In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, asin
from sklearn.preprocessing import StandardScaler
from math import radians, cos, sin, asin, sqrt

In [2]:
df_weather = pd.read_csv('weather.csv')
df_spray = pd.read_csv('spray.csv')
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

In [3]:
#parse date into year, month and day
def parsedate(df):
    df = df.copy()
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day'] = pd.DatetimeIndex(df['Date']).day
    return df   

df_weather = parsedate(df_weather)
df_spray = parsedate(df_spray)
df_test = parsedate(df_test)
df_train = parsedate(df_train)

In [4]:
#Nearest Neighbour analysis with Haversine Distance

def dist(lat1, lon1, lat2, lon2): 
    R = 6371
    dLat = np.radians(lat2 - lat1)
    dLon = np.radians(lon2 - lon1)
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    a = np.power(np.sin(dLat/2), 2) + np.multiply(np.cos(lat1), np.multiply(np.cos(lat2), np.power(np.sin(dLon/2), 2)))
    c = 2*np.arcsin(np.sqrt(a))
    km = R * c
    return km

def station1or2(df):
    df['lat1']=41.995  
    df['lat2']=41.786   
    df['lon1']=-87.933  
    df['lon2']=-87.752  
    df['dist1'] = dist(df.Latitude.values, df.Longitude.values, df.lat1.values, df.lon1.values) 
    df['dist2'] = dist(df.Latitude.values, df.Longitude.values, df.lat2.values, df.lon2.values)
    indicator = np.less_equal(df.dist1.values, df.dist2.values) 
    df_station = np.ones(df.shape[0])
    df_station[indicator==0]=2
    df['Station']=df_station     
    df.drop(['dist1', 'dist2', 'lat1', 'lat2', 'lon1', 'lon2' ], axis=1, inplace=True)
    return df

df_train = station1or2(df_train)
df_test = station1or2(df_test)

In [5]:
#Drop useless features

df_train = df_train.drop(['Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy', 'NumMosquitos'], axis =1)
df_test = df_test.drop(['Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy'], axis =1)

In [6]:
#cleaning Weather 

df_weather.replace(['M','-'], [np.nan, np.nan], inplace=True)

In [7]:
#drop features with too many nan values

df_weather  = df_weather.drop(['Water1', 'SnowFall', 'Depth', 'CodeSum'], axis =1)

In [8]:
#fill sensable missing data in station 2 with values from station 1

dfs = df_weather[['Depart', 'Sunset', 'Sunrise']].fillna(method = 'ffill')
df_weather = df_weather.drop( columns = ['Depart', 'Sunset', 'Sunrise'])
df_weather = pd.concat([df_weather, dfs], axis=1)

df_weather.isnull().sum()

Station         0
Date            0
Tmax            0
Tmin            0
Tavg           11
DewPoint        0
WetBulb         4
Heat           11
Cool           11
PrecipTotal     2
StnPressure     4
SeaLevel        9
ResultSpeed     0
ResultDir       0
AvgSpeed        3
Year            0
Month           0
Day             0
Depart          0
Sunset          0
Sunrise         0
dtype: int64

In [9]:
#replace nan for Traces of precipitation

df_weather.replace(['  T'], [0.01], inplace = True)

In [10]:
#replace nan value of Heat and cool missing in station 2 with station 1 values

dfa = df_weather[['Heat', 'Cool']].fillna(method = 'ffill')
df_weather = df_weather.drop( columns = ['Heat', 'Cool'])
df_weather = pd.concat([df_weather, dfa], axis=1)

In [11]:
#replace nan value of missing values with avergers with median

df_weather = df_weather.fillna(df_weather.median())

C:\Users\get gd nub\AppData\Local\Temp\ipykernel_14168\1028224986.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_weather = df_weather.fillna(df_weather.median())


In [12]:
#dealing with duplicates in data

df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()

In [13]:
#get dummies Species

df_train.Species.unique()
df_train.Species.value_counts()

dummies_train = pd.get_dummies(df_train['Species'])
df_train = pd.concat([df_train, dummies_train], axis=1)

dummies_test = pd.get_dummies(df_test['Species'])
df_test = pd.concat([df_test, dummies_test], axis=1)

In [14]:
#get dummies Trap

df_train.Trap.unique()
df_train.Trap.value_counts()

dummies_train = pd.get_dummies(df_train['Trap'])
df_train = pd.concat([df_train, dummies_train], axis=1)

dummies_test = pd.get_dummies(df_test['Trap'])
df_test = pd.concat([df_test, dummies_test], axis=1)

In [15]:
#merge weather and train/ test

df_train_merged = df_train.merge(df_weather, on=['Date', 'Station'], how="left")

df_test_merged = df_test.merge(df_weather, on=['Date', 'Station'], how="left")

In [16]:
#drop useless features

df_train_merged  = df_train_merged.drop(['Species', 'Trap', 'Date'], axis =1)
df_test_merged  = df_test_merged.drop(['Species', 'Trap', 'Date'], axis =1)

df_test_merged.to_csv("df_test_merged.csv")
df_train_merged.to_csv("df_train_merged.csv")